In [ ]:
from importlib import import_module
import os
import sys
import argparse
import linecache
import uproot
import vector
import math
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
from tqdm import tqdm  # ✅ 진행률 표시
import glob
import json
vector.register_awkward()


## HLT 를 먼저 통과 한 이벤트 , hlthps tau 0개 이상인 이벤트만 -> 남은 이벤트 중에서 gen match -> genmatch 가 된 hlthps 의 이벤트를 남겨 놓음  



def get_info (sample):
    file = uproot.open(sample)
    events = file["Events"]
    runs = file["Runs"]
    keys = events.keys()

    # 변수 불러오기 

    tau_pt = events["hltHpsPFTau_pt"].array()
    tau_eta = events["hltHpsPFTau_eta"].array()
    tau_phi = events["hltHpsPFTau_phi"].array()
    tau_mass = events["hltHpsPFTau_mass"].array()


    genvis_tau_pt = events["GenVisTau_pt"].array()
    genvis_tau_eta = events["GenVisTau_eta"].array()
    genvis_tau_phi = events["GenVisTau_phi"].array()
    genvis_tau_mass = events["GenVisTau_mass"].array()
    #print(len(genvis_tau_pt),"gen vis tau total events")

    # 필터 설정

    ## trigger filter ( returns_num_den 에서 사용됨  )
    tau_trigger_filter = events["HLT_LooseDeepTauPFTauHPS180_L2NN_eta2p1"].array()

    ## hlthpstau 1개 이상 필터 

    tau_num = ak.num(tau_pt)
    no_tau = (tau_num == 0) # [ False, True, False, ... ] 2000개 
    has_tau = ~no_tau


    hlt_has_tau_filter = (has_tau) ## hlthps tau 1개 이상 있는 이벤트만 선택
    tau_triggerd_list = (hlt_has_tau_filter==True)

    ## final selected variables ( hlthps 있는 이벤트만 남김 )
    final_tau_pt = tau_pt[tau_triggerd_list]
    final_tau_eta = tau_eta[tau_triggerd_list]
    final_tau_phi = tau_phi[tau_triggerd_list]
    final_tau_mass = tau_mass[tau_triggerd_list]

    final_genvis_tau_pt = genvis_tau_pt[tau_triggerd_list]
    final_genvis_tau_eta = genvis_tau_eta[tau_triggerd_list]
    final_genvis_tau_phi = genvis_tau_phi[tau_triggerd_list]
    final_genvis_tau_mass = genvis_tau_mass[tau_triggerd_list]

    tau_trigger_filtered = tau_trigger_filter[tau_triggerd_list]

    total_events = len(tau_pt)
    print("total events:", total_events,"->","after trigger filter:",ak.sum(has_tau),"->","after reconstructed hlthps ,events remain",len(final_tau_pt))
    return events,final_tau_pt, final_tau_eta, final_tau_phi, final_tau_mass, final_genvis_tau_pt, final_genvis_tau_eta, final_genvis_tau_phi, final_genvis_tau_mass ,genvis_tau_pt , genvis_tau_eta , genvis_tau_phi ,tau_trigger_filtered


def deltaR(eta1,phi1,eta2,phi2):
    delta_eta = eta1 - eta2
    delta_phi = phi1 - phi2
    delta_phi = (delta_phi + math.pi) % (2 * math.pi) - math.pi
    return np.sqrt(delta_eta**2 + delta_phi**2)

def gen_match(tau_pt, tau_eta, tau_phi, genvis_tau_pt, genvis_tau_eta, genvis_tau_phi, deltaR_threshold=0.3,delta_pt_threshold=30):
    matched = [] ## gen 이벤트 인덱스 안에 들어있는 각 gen 입자의 매칭 여부 리스트 [[ True(event1 1 particle), False(event1 2 particle), ... ](event1), [ False, False, ... ](event2), ... ]
    events_flag = [] ## gen match 된 이벤트인지 아닌지 gen 이벤트의 bool 리스트 [ True(event1), False(event2), ... ]

    leading_matched = []
    subleading_matched = []
    both_matched = []

    for i in range(len(tau_pt)):# 전체 이벤트 갯수 반복 ( gen 이벤트 수와 동일해야 하므로 reco 로 디버그 차  설정 )
        tau_pt_i = tau_pt[i]
        tau_eta_i = tau_eta[i]
        tau_phi_i = tau_phi[i]
        genvis_tau_pt_i = genvis_tau_pt[i]
        genvis_tau_eta_i = genvis_tau_eta[i]
        genvis_tau_phi_i = genvis_tau_phi[i]
        
        match_flags = [] # matched 안에 넣기 위한 한 이벤트 의 매칭 여부 리스트 ( 각 입자의 매칭 여부가 여기에 저장 )
        has_genmatch = False
        leading_match = False
        subleading_match = False
        for j in range(len(genvis_tau_pt_i)):
            genvis_tau_pt_ik = genvis_tau_pt_i[j]
            genvis_tau_eta_ik = genvis_tau_eta_i[j]
            genvis_tau_phi_ik = genvis_tau_phi_i[j]
            
            is_matched = False
            for k in range(len(tau_pt_i)):
                tau_pt_ij = tau_pt_i[k]
                tau_eta_ij = tau_eta_i[k]
                tau_phi_ij = tau_phi_i[k]
                
                dR = deltaR(tau_eta_ij, tau_phi_ij, genvis_tau_eta_ik, genvis_tau_phi_ik)
                dpt = abs(tau_pt_ij - genvis_tau_pt_ik)
                
                if dR < deltaR_threshold and dpt < delta_pt_threshold:
                    is_matched = True
                    has_genmatch = True
                    if k == 0:
                        leading_match = True
                    elif k == 1:
                        subleading_match = True
                    break # 각 gen이 reco 중 하나라도 매칭되면 루프 탈출
            match_flags.append(is_matched) # gen 입자당 루프 돌면서 해당 입자가 매칭 되었는지 저장 
        if has_genmatch:
            events_flag.append(True) # 이벤트 자체가 매칭된 이벤트인지 저장 
        else:
            events_flag.append(False)
        matched.append(np.array(match_flags)) ## 이벤트 당 match_flags(입자가 매칭됐는지를 리스트형태로 저장 )


        if leading_match :
            if subleading_match :
                both_matched.append(True)
                leading_matched.append(False)
                subleading_matched.append(False)
            if not subleading_match :
                leading_matched.append(True)
                subleading_matched.append(False)
                both_matched.append(False)

        if not leading_match :
            if subleading_match :
                subleading_matched.append(True)
                leading_matched.append(False)
                both_matched.append(False)
            else :
                subleading_matched.append(False)
                leading_matched.append(False)
                both_matched.append(False)


    return both_matched,leading_matched , subleading_matched , matched ,events_flag


#sample = "/gv0/Users/achihwan/phase2/cmssw_16/condor/hltrun/step2_fullstack_180.root"
#events_180,final_tau_pt_180, final_tau_eta_180, final_tau_phi_180, final_tau_mass_180, final_genvis_tau_pt_180, final_genvis_tau_eta_180, final_genvis_tau_phi_180, final_genvis_tau_mass_180 ,genvis_tau_pt_180 , genvis_tau_eta_180, genvis_tau_phi_180,tau_trigger_filter_180= get_info(sample)
#both_matched_180, leading_only_matched_180 , subleading_only_matched_180,gen_matched_180,events_flag_180 = gen_match(final_tau_pt_180, final_tau_eta_180, final_tau_phi_180, final_genvis_tau_pt_180, final_genvis_tau_eta_180, final_genvis_tau_phi_180)


sample = "/gv0/Users/achihwan/phase2/cmssw_16/condor/hltrun/step2_fullstack_150.root"
events_150,final_tau_pt_150, final_tau_eta_150, final_tau_phi_150, final_tau_mass_150, final_genvis_tau_pt_150, final_genvis_tau_eta_150, final_genvis_tau_phi_150, final_genvis_tau_mass_150,genvis_tau_pt_150 , genvis_tau_eta_150, genvis_tau_phi_150,tau_trigger_filter_150 = get_info(sample)
both_matched_150, leading_only_matched_150 , subleading_only_matched_150,gen_matched_150,events_flag_150 = gen_match(final_tau_pt_150, final_tau_eta_150, final_tau_phi_150, final_genvis_tau_pt_150, final_genvis_tau_eta_150, final_genvis_tau_phi_150)




In [ ]:

import cmsstyle as CMS
import mplhep as hep
import matplotlib.pyplot as plt

## 리딩 서브리딩 아무거나 매칭 된 경우 
def returns_num_den (gen_matched, events_flag, final_tau_pt, final_tau_eta, final_tau_phi, final_genvis_tau_pt, final_genvis_tau_eta, final_genvis_tau_phi, genvis_tau_pt, genvis_tau_eta, genvis_tau_phi,tau_trigger_filter):
    gen_matched = ak.Array(gen_matched) ## 매치된 hlthpspftau 필터 
    genmatched_flags = gen_matched[events_flag] ## 매치된 이벤트중에 매치된 입자들 
    print("after gen match , events remain:",len(genmatched_flags))
    
    match_genvistau_pt = final_genvis_tau_pt[events_flag] # 매치된 이벤트만  살리는 필터 적용 
    match_genvistau_pt = ak.sort(match_genvistau_pt, ascending=False)
    match_genvistau_eta = final_genvis_tau_eta[events_flag]
    match_genvistau_phi = final_genvis_tau_phi[events_flag]

    #using_eta = (abs(match_genvistau_eta)<2.1)
    #print(match_genvistau_eta)

    match_trigger_filter = events_flag & tau_trigger_filter #& using_eta

    triggered_genvistau_pt = final_genvis_tau_pt[match_trigger_filter]
    triggered_genvistau_pt = ak.sort(triggered_genvistau_pt, ascending=False)
    triggered_genvistau_eta = final_genvis_tau_eta[match_trigger_filter]
    triggered_genvistau_phi = final_genvis_tau_phi[match_trigger_filter]
    dummy = 1
    dummy2 = 2
    num_pt = triggered_genvistau_pt
    num_eta = triggered_genvistau_eta
    num_phi = triggered_genvistau_phi
    ## denominator ( 모든 genvistau 정보 )
    den_pt = match_genvistau_pt
    den_eta = match_genvistau_eta
    den_phi = match_genvistau_phi
    return num_pt, den_pt, num_eta, den_eta, num_phi, den_phi
num_pt_150 , den_pt_150, num_eta_150, den_eta_150, num_phi_150, den_phi_150 = returns_num_den (gen_matched_150, events_flag_150, final_tau_pt_150, final_tau_eta_150, final_tau_phi_150, final_genvis_tau_pt_150, final_genvis_tau_eta_150, final_genvis_tau_phi_150, genvis_tau_pt_150, genvis_tau_eta_150, genvis_tau_phi_150,tau_trigger_filter_150)
#pt 150 filter

# leading tau
num_pt_1501 = num_pt_150 [:,0]
den_pt_1501 = den_pt_150 [:,0]
num_eta_150 = num_eta_150 [:,0]
den_eta_150 = den_eta_150 [:,0]
num_phi_150 = num_phi_150 [:,0]
den_phi_150 = den_phi_150 [:,0]

den_eta_filter = (abs(den_eta_150) < 2.1)
num_eta_filter = (abs(num_eta_150) < 2.1)


num_pt_1501 = num_pt_1501 [num_eta_filter]
den_pt_1501 = den_pt_1501 [den_eta_filter]
###Pt plot





bins = np.arange(0, 500 , 30)


num_pt_1501 = ak.to_numpy(num_pt_1501)
den_pt_1501 = ak.to_numpy(den_pt_1501)
counts_num_150, edges = np.histogram(num_pt_1501, bins=bins)
counts_den_150, _ = np.histogram(den_pt_1501, bins=bins)

ratio_150 = np.divide(counts_num_150, counts_den_150, 
                      out=np.zeros_like(counts_num_150, dtype=float), 
                      where=(counts_den_150 != 0))
ratio_err_150 = np.zeros_like(ratio_150)
nonzero_150 = counts_den_150 > 0
ratio_err_150[nonzero_150] = np.sqrt(ratio_150[nonzero_150] * (1.0 - ratio_150[nonzero_150]) / counts_den_150[nonzero_150])


# plot with error bars
bin_centers = (edges[:-1] + edges[1:]) / 2.0
half_widths = (edges[1:] - edges[:-1]) / 2.0

plt.style.use(hep.style.CMS)
fig, ax = plt.subplots(figsize=(10, 8))

# 두 데이터셋 그리기
ax.errorbar(bin_centers, ratio_150, xerr=half_widths, yerr=ratio_err_150, 
            fmt='o', color='C0', ecolor='C0', capsize=4, label='pT > 150 GeV')



ax.set_xlabel(r'Gen Tau $\eta$', fontsize=18)
ax.set_xlim(edges[0], edges[-1])
ax.set_ylim(0, 1.1)

# Legend 추가
ax.legend(loc='best', fontsize=14)

ax.grid(which='minor', linestyle=':', linewidth=0.5, alpha=0.5)

# CMS 스타일 레이블 추가
hep.cms.text("Simulation Preliminary", loc=2, ax=ax, fontsize=18)
ax.text(0.05, 0.85, r"$Z'(500GeV) \to \tau \tau$, PU=200",
    transform=ax.transAxes, fontsize=15, va='top', ha='left',
    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
ax.ticklabel_format(style="sci", scilimits=(-3, 3), useMathText=True)
ax.get_yaxis().get_offset_text().set_position((-0.085, 1.05))

plt.tight_layout()
plt.show()

